## Import libraries

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import os
import matplotlib.pyplot as plt

#reproducible
torch.manual_seed(1)

In [ ]:
use_cuda = torch.cuda.is_available()
print(use_cuda)
use_cuda = False

If the above output **False** means you can't use cuda acceleration. Don't worry It'll still work fine.

## Define LeNet CNN

**LeNet structure:**

Conv2d -> Activation -> MaxPooling

-> Conv2d -> Activation -> MaxPooling

-> Flattening -> FullyConnected -> Activation

-> FullyConnected -> Activation

-> FullyConnected -> Activation

![LeNet](https://pic1.zhimg.com/80/v2-e144b987285eb9cdba097c9375244074_hd.jpg)

Used function:
- Convolution: nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
- Activation: nn.ReLU()
- MaxPooling: nn.MaxPool2d(kernel_size)
- FullyConnected: nn.Linear(input_channels, output_channels)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        '''Comlete this part'''
        

    def forward(self, x):
        '''Comlete this part'''
        return output, x    # return x for visualization

In [ ]:
cnn = CNN()
if(use_cuda): cnn.cuda() # Moves all model parameters and buffers to the GPU.
print(cnn)

## Preparing Dataset

Check if MNIST is already downloaded

In [ ]:
DOWNLOAD_FM = False
# Mnist digits dataset
if not(os.path.exists('./dataset/FashionMNIST/')) or not os.listdir('./dataset/FashionMNIST/'):
    # not mnist dir or mnist is empyt dir
    DOWNLOAD_FM = True
    
print(DOWNLOAD_FM)

Loading training data

In [ ]:
# Data preprocessing: normalize image data so that grayscale data is between -1 and +1
tf = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.5,), (0.5,))])

train_data = torchvision.datasets.FashionMNIST(
    root='./dataset/',
    train=True,
    transform=tf,
    download=DOWNLOAD_FM,
)
# Data Loader for easy mini-batch return in training, the image batch shape will be (50, 1, 28, 28)
train_loader = Data.DataLoader(dataset=train_data, batch_size=64, shuffle=True)

pick 2000 samples to speed up testing

In [ ]:
test_data = torchvision.datasets.FashionMNIST(
    root='./dataset/',
    train=False,
    transform=tf
)
test_loader = Data.DataLoader(test_data, batch_size=64, shuffle=True)

In [ ]:
labellist = ['T-shirt','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']
# print(f'The corresponding label for this image is {labellist[imagedemolabel]}')

plot one example

In [ ]:
PIC_IDX = 800 # change this to see different picture
print(train_data.train_data.size())                 # (60000, 28, 28)
print(train_data.train_labels.size())               # (60000)
plt.imshow(train_data.train_data[PIC_IDX].numpy(), cmap='gray')
plt.title(labellist[train_data.train_labels[PIC_IDX]])
plt.show()

## Training
Now we are ready to train

In [ ]:
EPOCH = 5               # train the training data n times, to save time, we just train 1 epoch
LR = 0.01              # learning rate

optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)   # optimize all cnn parameters
loss_func = nn.CrossEntropyLoss()                       # the target label is not one-hotted

In [ ]:
# train_losses, test_losses = [], []

for epoch in range(EPOCH):
    for step, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        if(use_cuda):
            images = images.cuda()
            labels = labels.cuda()
        log_ps, hidden = cnn(images)
        loss = loss_func(log_ps, labels)
        loss.backward()
        optimizer.step()
    
        if step%100 == 0 or step == len(train_loader)-1:
            test_loss = 0
            accuracy = 0
            with torch.no_grad():
                cnn.eval()
                for images, labels in test_loader:
                    if(use_cuda):
                        images = images.cuda()
                        labels = labels.cuda()
                    log_ps, hidden = cnn(images)
                    test_loss += loss_func(log_ps, labels)
                    ps = torch.exp(log_ps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))
                if(use_cuda):
                    train_loss = loss.data.cpu()
                    validation_loss = test_loss/len(test_loader)
                    acc = accuracy/len(test_loader)
                else:
                    train_loss = loss.data.numpy()
                    validation_loss = test_loss/len(test_loader)
                    acc = accuracy/len(test_loader)
                    
                print("epoch: {}/{}| ".format(epoch+1, EPOCH),
                      "step: {}/{}|".format(step,len(train_loader)),
                      "train loss: {:.3f}| ".format(train_loss),
                      "Validation loss: {:.3f}| ".format(validation_loss),
                      "accuracy: {:.3f}".format(acc))
                cnn.train()
cnn.eval()